In [1]:
import pandas as pd

mgf_file = "/mnt/data/cdens/casanovo-scaling/massivekb_data/massiveKB_3cac03860ff7453a821332ab4cff20f4.mgf"
metadata_file = "/mnt/data/cdens/casanovo-scaling/massivekb_data/LIBRARY_AUGMENT-3cac0386-candidate_library_spectra-main.tsv"

metadata = pd.read_csv(metadata_file, sep="\t")
metadata

,rowid,annotation,augment_task,charge,explained_intensity,extract_task,filename,fragment_tolerance,fragmentation_method,most_similar_score,ppm_error,proteosafe_task,scan,score
0,1,-17.027QAAAAAAAAAATAAWTGTTF,3cac03860ff7453a821332ab4cff20f4,2,1.000000,d62b293e053548729ecc1f01c5e374e5,MSV000080813/ccms_peak/RAW/20130504_EXQ6_SaDe_...,0.05,HCD,0.699075,1.715936,b22fa27763ad42b9be1f8f08d5ee86e5,88883,2.587908
1,2,-17.027QAAAAAAAAAATAAWTGTTF,3cac03860ff7453a821332ab4cff20f4,2,0.906294,d62b293e053548729ecc1f01c5e374e5,MSV000080056/ccms_peak/RAW/20110819_EXQ4_TaGe_...,0.05,HCD,0.705949,2.461911,129e2d62c17b4b86b2ba166cb35dbd42,86975,2.053339
2,3,-17.027QAAAAAAAAAATAAWTGTTF,3cac03860ff7453a821332ab4cff20f4,2,0.934026,d62b293e053548729ecc1f01c5e374e5,MSV000080813/ccms_peak/RAW/20130429_EXQ6_SaDe_...,0.05,HCD,0.718899,2.729332,60d9969c20c1489390974c8a35efbfd9,90120,2.177906
3,4,-17.027QAAAAAAAAAATAAWTGTTF,3cac03860ff7453a821332ab4cff20f4,2,0.466789,b014c8098e2f459a95e6ceb167755f23,MSV000083952/ccms_peak/RAW/M411-D09-O156-HS-P4...,0.40,CID,0.181958,3.523141,c368e69068334b1196d26d849d92ee6c,13208,0.186828
4,5,-17.027QAAAAAAAAAATAAWTGTTF,3cac03860ff7453a821332ab4cff20f4,2,0.884540,d62b293e053548729ecc1f01c5e374e5,MSV000080813/ccms_peak/RAW/20130425_EXQ6_SaDe_...,0.05,HCD,0.721200,2.203290,b22fa27763ad42b9be1f8f08d5ee86e5,93437,1.672937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66184183,66184184,YYYYWHLRK,3cac03860ff7453a821332ab4cff20f4,4,0.577963,d62b293e053548729ecc1f01c5e374e5,MSV000080812/ccms_peak/RAW/QE6_LC6_IAH_SUMO_U2...,0.05,HCD,0.553644,1.815275,1310554b180c4e3486a2defc742982bc,9803,5.857590
66184184,66184185,YYYYYMDVWGK,3cac03860ff7453a821332ab4cff20f4,2,0.577045,d62b293e053548729ecc1f01c5e374e5,MSV000080835/ccms_peak/RAW/Animal_20141209_MH_...,0.05,HCD,0.908541,0.115215,d38b024e34584b8f8e33878d24b99e4e,70545,6.703461
66184185,66184186,YYYYYMDVWGK,3cac03860ff7453a821332ab4cff20f4,2,0.481196,d62b293e053548729ecc1f01c5e374e5,MSV000080368/ccms_peak/StanleyJ-15-G042-PV3A-B...,0.05,HCD,0.951414,1.724344,0069db4e3aeb496f8896abd04cb914dc,27676,6.770455
66184186,66184187,YYYYYMDVWGK,3cac03860ff7453a821332ab4cff20f4,2,0.516306,d62b293e053548729ecc1f01c5e374e5,MSV000080368/ccms_peak/StanleyJ-15-G042-PV3A-B...,0.05,HCD,0.964591,2.131604,0bfa1189b457424ba2b14c30418986f7,28099,7.192316


In [2]:
# Creating a column with filename only for faster searching
metadata['mz_file'] = metadata['filename'].str.split('/').str[-1]

In [3]:
# Using mz_file scan multiindex for much faster lookup
metadata.set_index(['mz_file', 'scan'], inplace=True)
metadata = metadata.sort_index()

In [5]:
import os
from tqdm import tqdm
from pyteomics import mgf

chunk_size = 1000000
chunk_index = 0
chunk_dir = "/mnt/data/cdens/casanovo-scaling/massivekb_data/chunks/"
os.makedirs(chunk_dir, exist_ok=True)

chunk = []
with mgf.read(
        mgf_file,
        use_index=False,
        convert_arrays=0,
        read_charges=False,
        read_ions=False,
) as massivekb:
    for spectrum in tqdm(massivekb, total=66e6):
        if "charge" not in spectrum['params']:
            mzml_file, _, scan = spectrum['params']['title'].split(':')
            metadata_rows = metadata.loc[(mzml_file, int(scan))]
            if len(metadata_rows) == 0:
                print(f"Did not find metadata for {mzml_file}")
            elif len(metadata_rows) > 1:
                print(f"Found multiple metadata for {mzml_file}")
                print(metadata_rows['filename'])
            else:
                charge = metadata_rows.iloc[0]['charge']
                spectrum['params']['charge'] = charge
        chunk.append(spectrum)
        if len(chunk) >= chunk_size:
            out_file = os.path.join(chunk_dir, f"chunk_{chunk_index}.mgf")
            mgf.write(chunk, out_file, use_numpy=True)
            chunk_index += 1
            chunk = []
    if chunk:
        out_file = os.path.join(chunk_dir, f"chunk_{chunk_index}.mgf")
        mgf.write(chunk, out_file, use_numpy=True)

  3%|▎         | 1696626/66000000.0 [10:33<6:39:56, 2679.70it/s] 


KeyboardInterrupt: 